Python possui recursos nativos para manipulação de strings. No entanto, buscar e até mesmo substituir trechos de strings que possam atender a mais de um tipo de padrão pode ser uma tarefa árdua e pouco viável em termos práticos utilizando somente métodos built-in. Por isso, as expressões regulares, também conhecidas como RegEx, surgiram para facilitar esses processos.

Dentro da documentação oficial de RegEx, em tradução livre: "Os padrões de expressões regulares são compilados em uma série de bytecodes que são depois executados por um motor escrito em C". A utilização de C justifica também maior eficiência na execução de buscas. Além disso, a documentação recomenda para casos em que a utilização de RegEx fique muito complexa, que sejam criadas funções específicas para tal finalidade, mesmo que isso possa implicar uma eficiência inferior.

A elaboração deste notebook tem como base a documentação do módulo de expressões regulares, que está disponível [aqui](https://docs.python.org/3/howto/regex.html).

Neste notebook serão utilizadas sentenças definidas em células e também um arquivo .txt com parte da [página do curso de Ciência de Dados do site do ICMC](https://icmc.usp.br/graduacao/ciencia-de-dados-bacharelado).

In [ ]:
import pandas as pd
import numpy as np
import re

# Split

Dentre os métodos nativos, existia o split(). A biblioteca re também oferece recursos para efetuar o split de sentenças.

In [ ]:
sentence = 'Quem nasce no Brasil é Br.'
re.split(' ', sentence)

['Quem', 'nasce', 'no', 'Brasil', 'é', 'Br.']

In [ ]:
re.split('Br', sentence)

['Quem nasce no ', 'asil é ', '.']

As expressões regulares conseguem fazer essas separações de tokens mesmo que a regra se aplique a sequências de caracteres entendidas como palavras.

# Buscas

É possível também realizar buscas dentro de uma string (aplicável a arquivos .txt, células de dataframes ou arrays, entre outros). O local da primeira ocorrência é informado.

In [ ]:
re.search('Brasil', sentence)

<re.Match object; span=(14, 20), match='Brasil'>

In [ ]:
re.search('as', sentence)

<re.Match object; span=(6, 8), match='as'>

Para saber o número de ocorrências, basta utilizar outro recurso, o 'findall'.

In [ ]:
re.findall('Brasil', sentence)

['Brasil']

In [ ]:
re.findall('as', sentence)

['as', 'as']

No entanto, para encontrar o local de ocorrência de todos esses elementos, é necessário iterar dentro da string de interesse com finditer() ou utilizar um loop for com um iterator.

In [ ]:
for i in re.finditer('as', sentence):
  print(i)

<re.Match object; span=(6, 8), match='as'>
<re.Match object; span=(16, 18), match='as'>


In [ ]:
iterator = re.finditer('as', sentence)
list_occurrence = []
for i in iterator:
  print(i.span())
  list_occurrence.append(i)

(6, 8)
(16, 18)


In [ ]:
print('Lista completa de ocorrências: {}'.format(list_occurrence))
print('O tipo do primeiro elemento {} é: {}'.format(list_occurrence[0],type(list_occurrence[0])))
print('O tipo do primeiro span {} é: {}'.format(list_occurrence[0].span(),type(list_occurrence[0].span())))
print('O tipo do primeiro elemento {} do primeiro span é: {}'.format(list_occurrence[0].span()[0],type(list_occurrence[0].span()[0])))

Lista completa de ocorrências: [<re.Match object; span=(6, 8), match='as'>, <re.Match object; span=(16, 18), match='as'>]
O tipo do primeiro elemento <re.Match object; span=(6, 8), match='as'> é: <class 're.Match'>
O tipo do primeiro span (6, 8) é: <class 'tuple'>
O tipo do primeiro elemento 6 do primeiro span é: <class 'int'>


# Meta-caracteres

Dentro do contexto de RegEx, existem caracteres especiais que são denominados meta-caracteres. Tais caracteres podem realizar operações específicas de busca e substituição.

Eles incluem:

. ^ $ * + ? { } [ ] \ | ( )

Quando os meta-caracteres são representados entre colchetes [] eles deixam de realizar operações e passam a representar possíveis correspondências no texto. Por exemplo, [*] buscará por caracteres * que estejam dentro do texto.

A seguir serão exemplificados os meta-caracteres.

## Ponto (.)

In [ ]:
string_meta = 'lista de metacaracteres . ^ $ * + ? { } [ ] \ | ( )'
# O . é capaz de dar "match" com qualquer caractere, exceto caractere de nova linha. Além disso, o [.] mostra que o ponto final deixa de ser um operador entre colchetes
print(re.search('.',string_meta))
print(re.search('[.]',string_meta))

<re.Match object; span=(0, 1), match='l'>
<re.Match object; span=(24, 25), match='.'>


## Circunflexo (^)

In [ ]:
string_meta = 'lista de metacaracteres . ^ $ * + ? { } [ ] \ | ( )'

# O ^ significa "exceto" quando no início de um colchete. Por exemplo, na linha a seguir o resultado retornado será o primeiro que não seja a letra l minúscula
print(re.search('[^l]',string_meta))

# No caso a seguir, o caractere representará o próprio acento. Os outros caracteres não necessariamente ficam na mesma ordem apresentada nos colchetes.
string_meta = 'lista ^ 2de metacaracteres . ^ $ * + ? { } [ ] \ | ( )'
print(re.findall('[2^]',string_meta))

<re.Match object; span=(1, 2), match='i'>
['^', '2', '^']


## Cifrão ($)

In [ ]:
string_meta = 'lista de metacaracteres . ^ $ * + ? { } [ ] \ | ( )'

# O $ significa que a string em análise deve ter como padrão final a expressão que antecede $. Será vazio, pois a string não termina com 'a'
print(re.search('a$',string_meta))
print(re.search('[$]',string_meta))

None
<re.Match object; span=(28, 29), match='$'>


## Asterisco (*)

Na string, a partir desta seção, foram incluídos risos (haha) com diferentes números de caracteres, a fim de refletir algo comumente encontrado em textos de redes sociais como o Twitter. O pré-processamento desses textos é um dos desafios de quem trabalha com processamento de linguagem natural, em particular com textos gerados por usuários (user-generated content, UGC), que são mais informais.

In [ ]:
string_meta = 'haha haahaa haaaahaaaa h lista de metacaracteres . ^ $ * + ? { } [ ] \ | ( )'

# O * pode capturar zero ou mais ocorrências de uma dada expressão
print(re.findall('ha*',string_meta))
print(re.search('[*]',string_meta))

['ha', 'ha', 'haa', 'haa', 'haaaa', 'haaaa', 'h']
<re.Match object; span=(55, 56), match='*'>


## Mais (+)

In [ ]:
string_meta = 'haha haahaa haaaahaaaa h lista de metacaracteres . ^ $ * + ? { } [ ] \ | ( )'

# O + significa que a parte de interesse da expressão é repetida no mínimo uma vez
print(re.findall('ha+',string_meta))
print(re.search('[+]',string_meta))

['ha', 'ha', 'haa', 'haa', 'haaaa', 'haaaa']
<re.Match object; span=(57, 58), match='+'>


## Interrogação (?)

In [ ]:
string_meta = 'haha haahaa haaaahaaaa h lista de metacaracteres . ^ $ * + ? { } [ ] \ | ( )'

# O ? significa que a parte de interesse da expressão não ocorre ou ocorre no máximo uma vez
print(re.findall('ha?',string_meta))
print(re.search('[?]',string_meta))

['ha', 'ha', 'ha', 'ha', 'ha', 'ha', 'h']
<re.Match object; span=(59, 60), match='?'>


## Chaves ({ })

In [ ]:
string_meta = 'haha haahaa hhaaaahaaaa h lista de metacaracteres . ^ $ * + ? { } [ ] \ | ( )'

# O {} significa que a parte de interesse da expressão deve ocorrer em um número pré-definido de vezes. O h deve ser seguido por 'a' ou 'aa'
print(re.findall('ha{1,2}',string_meta))
print(re.findall('[{}]',string_meta)) # search somente retorna a primeira ocorrência, por isso o findall

['ha', 'ha', 'haa', 'haa', 'haa', 'haa']
['{', '}']


##Colchetes ([ ])

In [ ]:
string_meta = 'haha haahaa hhaaaahaaaa h lista de metacaracteres . ^ $ * + ? { } [ ] \ | ( )'

# O [] faz com que o que esteja dentro dele deixe de ser um metacaractere. 
print(re.findall('[{}]',string_meta))
# O [] também é responsável por aplicar certas operações somente aos elementos dentro dele. No caso, um dos elementos dentro do colchete deve ocorrer pelo menos uma vez.
print(re.findall('[ha]+',string_meta))
# Neste caso, o h deve sempre ter depois dele pelo menos uma letra a
print(re.findall('h[a]+',string_meta))

['{', '}']
['haha', 'haahaa', 'hhaaaahaaaa', 'h', 'a', 'a', 'a', 'a']
['ha', 'ha', 'haa', 'haa', 'haaaa', 'haaaa']


## Barra Vertical (|)

In [ ]:
string_meta = 'haha haahaa hhaaaahaaaa h lista de metacaracteres . ^ $ * + ? { } [ ] \ | ( )'

# O | significa "ou" dentro da expressão regular
print(re.findall('de|lista',string_meta))
print(re.findall('[|]',string_meta))

['lista', 'de']
['|']


## Parênteses ( ( ) )

In [ ]:
string_meta = 'haha haahaa hhaaaahaaaa h lista de metacaracteres . ^ $ * + ? { } [ ] \ | ( ) 1'

# () são utilizados para encontrar sequências de caracteres idênticas às da expressão
print(re.findall('(lista)',string_meta))
print(re.findall('lista',string_meta))

# Embora as duas expressões anteriores tenham os mesmos resultados, () são utilizados em outros contextos, que serão exemplificados depois do metacaractere '\'
print(re.findall('[()]',string_meta))

['lista']
['lista']
['(', ')']


## Barra Inverta "Backslash" ( \ )

In [ ]:
string_meta = 'haha haahaa hhaaaahaaaa h lista de metacaracteres . ^ $ * + ? { } [ ] \ | ( )'
# O \ faz com que o que esteja imediatamente depois dele deixe de ser um metacaractere.
print(re.search('\*',string_meta))

<re.Match object; span=(56, 57), match='*'>


O ' \ ' também permite a utilização de sequências especiais

In [ ]:
string_meta = 'haha haahaa hhaaaahaaaa h lista de metacaracteres . ^ $ * + ? { } [ ] \ | ( ) 0 1 2 3 \n \t'

# \d - qualquer caractere que seja numérico
print(re.findall('\d',string_meta))
# equivalente usando colchetes
print(re.findall('[0-9]',string_meta))

['0', '1', '2', '3']
['0', '1', '2', '3']


In [ ]:
# \D - qualquer caractere que NÃO seja numérico
print(re.findall('\D',string_meta))
# equivalente usando colchetes
print(re.findall('[^0-9]',string_meta))

['h', 'a', 'h', 'a', ' ', 'h', 'a', 'a', 'h', 'a', 'a', ' ', 'h', 'h', 'a', 'a', 'a', 'a', 'h', 'a', 'a', 'a', 'a', ' ', 'h', ' ', 'l', 'i', 's', 't', 'a', ' ', 'd', 'e', ' ', 'm', 'e', 't', 'a', 'c', 'a', 'r', 'a', 'c', 't', 'e', 'r', 'e', 's', ' ', '.', ' ', '^', ' ', '$', ' ', '*', ' ', '+', ' ', '?', ' ', '{', ' ', '}', ' ', '[', ' ', ']', ' ', '\\', ' ', '|', ' ', '(', ' ', ')', ' ', ' ', ' ', ' ', ' ', '\n', ' ', '\t']
['h', 'a', 'h', 'a', ' ', 'h', 'a', 'a', 'h', 'a', 'a', ' ', 'h', 'h', 'a', 'a', 'a', 'a', 'h', 'a', 'a', 'a', 'a', ' ', 'h', ' ', 'l', 'i', 's', 't', 'a', ' ', 'd', 'e', ' ', 'm', 'e', 't', 'a', 'c', 'a', 'r', 'a', 'c', 't', 'e', 'r', 'e', 's', ' ', '.', ' ', '^', ' ', '$', ' ', '*', ' ', '+', ' ', '?', ' ', '{', ' ', '}', ' ', '[', ' ', ']', ' ', '\\', ' ', '|', ' ', '(', ' ', ')', ' ', ' ', ' ', ' ', ' ', '\n', ' ', '\t']


In [ ]:
# \s qualquer caractere que seja um espaço (incluindo novas linhas, por exemplo)
print(re.findall('\s',string_meta))
# equivalente usando colchetes
print(re.findall('[ \t\n\r\f\v]',string_meta))

[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '\n', ' ', '\t']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '\n', ' ', '\t']


In [ ]:
# \S qualquer caractere que NÃO SEJA um espaço (incluindo novas linhas, por exemplo)
print(re.findall('\S',string_meta))
# equivalente usando colchetes
print(re.findall('[^ \t\n\r\f\v]',string_meta))

['h', 'a', 'h', 'a', 'h', 'a', 'a', 'h', 'a', 'a', 'h', 'h', 'a', 'a', 'a', 'a', 'h', 'a', 'a', 'a', 'a', 'h', 'l', 'i', 's', 't', 'a', 'd', 'e', 'm', 'e', 't', 'a', 'c', 'a', 'r', 'a', 'c', 't', 'e', 'r', 'e', 's', '.', '^', '$', '*', '+', '?', '{', '}', '[', ']', '\\', '|', '(', ')', '0', '1', '2', '3']
['h', 'a', 'h', 'a', 'h', 'a', 'a', 'h', 'a', 'a', 'h', 'h', 'a', 'a', 'a', 'a', 'h', 'a', 'a', 'a', 'a', 'h', 'l', 'i', 's', 't', 'a', 'd', 'e', 'm', 'e', 't', 'a', 'c', 'a', 'r', 'a', 'c', 't', 'e', 'r', 'e', 's', '.', '^', '$', '*', '+', '?', '{', '}', '[', ']', '\\', '|', '(', ')', '0', '1', '2', '3']


In [ ]:
# \w qualquer caractere que seja alfanumérico
print(re.findall('\w',string_meta))
# equivalente usando colchetes
print(re.findall('[a-zA-Z0-9]',string_meta))

['h', 'a', 'h', 'a', 'h', 'a', 'a', 'h', 'a', 'a', 'h', 'h', 'a', 'a', 'a', 'a', 'h', 'a', 'a', 'a', 'a', 'h', 'l', 'i', 's', 't', 'a', 'd', 'e', 'm', 'e', 't', 'a', 'c', 'a', 'r', 'a', 'c', 't', 'e', 'r', 'e', 's', '0', '1', '2', '3']
['h', 'a', 'h', 'a', 'h', 'a', 'a', 'h', 'a', 'a', 'h', 'h', 'a', 'a', 'a', 'a', 'h', 'a', 'a', 'a', 'a', 'h', 'l', 'i', 's', 't', 'a', 'd', 'e', 'm', 'e', 't', 'a', 'c', 'a', 'r', 'a', 'c', 't', 'e', 'r', 'e', 's', '0', '1', '2', '3']


In [ ]:
# \W qualquer caractere que NÃO SEJA alfanumérico
print(re.findall('\W',string_meta))
# equivalente usando colchetes
print(re.findall('[^a-zA-Z0-9]',string_meta))

[' ', ' ', ' ', ' ', ' ', ' ', ' ', '.', ' ', '^', ' ', '$', ' ', '*', ' ', '+', ' ', '?', ' ', '{', ' ', '}', ' ', '[', ' ', ']', ' ', '\\', ' ', '|', ' ', '(', ' ', ')', ' ', ' ', ' ', ' ', ' ', '\n', ' ', '\t']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', '.', ' ', '^', ' ', '$', ' ', '*', ' ', '+', ' ', '?', ' ', '{', ' ', '}', ' ', '[', ' ', ']', ' ', '\\', ' ', '|', ' ', '(', ' ', ')', ' ', ' ', ' ', ' ', ' ', '\n', ' ', '\t']


In [ ]:
# \A verifica a expressão somente no início da string
print(re.search('\Ah',string_meta))
print(re.search('\Ab',string_meta))

<re.Match object; span=(0, 1), match='h'>
None


In [ ]:
# \b delimita a fronteira da palavra

string_meta = 'haha haahaa hhaaaahaaaa h lista de metacaracteres . ^ $ * + ? { } [ ] \ | ( )'
print(re.search(r'\blista\b',string_meta))
print(re.search(r'\blist\b',string_meta))

<re.Match object; span=(26, 31), match='lista'>
None


In [ ]:
# Podem ser também combinados
print(re.findall('[\W 1 2]',string_meta))

[' ', ' ', ' ', ' ', ' ', ' ', ' ', '.', ' ', '^', ' ', '$', ' ', '*', ' ', '+', ' ', '?', ' ', '{', ' ', '}', ' ', '[', ' ', ']', ' ', '\\', ' ', '|', ' ', '(', ' ', ')']


# Seleção de trechos (non-capturing)

Lookahead e lookbehind são utilizados quando é necessário verificar se a condição é verdadeira depois ou antes do trecho de input da string de interesse, respectivamente.

Com a expressão lookahead positiva o que está imediatamente anterior só é exibido caso a expressão que esteja dentro dos parênteses e depois do (?=...) seja válida.

Com o lookahead negativo (?!...), só haverá um retorno não vazio caso a expressão que está depois de ?! seja falsa.

In [ ]:
#string_non = 'lista de metacaracteres . ^ $ * + ? { } [ ] \ | ()'
string_non = 'abcdefghij'

# Lookahead positivo (?=...)
print('Lookahead positivo')
print(re.search(r'j(?=[a-z])',string_non))
print(re.search(r'i(?=[a-z])',string_non))
print(re.search(r'i(?=[a-z]{2})',string_non))

print('\n')
print('Lookahead negativo')
# Lookahead negativo (?!...).
print(re.search(r'j(?![a-z])',string_non))
print(re.search(r'i(?![a-z])',string_non))
print(re.search(r'i(?![a-z]{2})',string_non))

Lookahead positivo
None
<re.Match object; span=(8, 9), match='i'>
None


Lookahead negativo
<re.Match object; span=(9, 10), match='j'>
None
<re.Match object; span=(8, 9), match='i'>


O trecho códigos de lookbehind tem como base [este link](https://www.geeksforgeeks.org/python-regex-lookbehind/). Essa expressão verifica o que está depois (à direita na string) do que foi incluído dentro dos parênteses.

In [ ]:
string_non = 'ciencia de dados'

# Lookbehind (?<=...)
print('Lookbehind')
print(re.findall(r'(?<=e)[a-z]*',string_non))
print(re.findall(r'(?<=a)[a-z]*',string_non))
print(re.findall(r'(?<=i)[a-z]*',string_non))
print(re.findall(r'(?<=d)[a-z]*',string_non))

Lookbehind
['ncia', '']
['', 'dos']
['encia']
['e', 'ados']


In [ ]:
print(re.search(r'i(?=[a-z])',string_non))

<re.Match object; span=(1, 2), match='i'>


# Grupos Nomeados

Em alguns cenários, é conveniente extrair expressões que sejam relacionadas a nomes, podendo ser em formato de dicionários. Para obter um grupo nomeado, deve-se iniciar a expressão por (?P< nomedogrupo > regra para obtenção dos grupos).

In [ ]:
string = 'Primeironome Últimosobrenome'

for item in re.finditer('(?P<Nome>[\w]*)[\s](?P<Sobrenome>[\w]*)',string):
  print(item.groupdict())

{'Nome': 'Primeironome', 'Sobrenome': 'Últimosobrenome'}


# Substituição

Um arquivo .txt que extraiu parte do HTML do site do ICMC será utilizado nesta seção para que seja possível aplicar as expressões que foram apresentadas acima.

In [ ]:
f = open("site_icmc_html.txt", "r")
aux_file = f.read()
f.close()
print(aux_file)

<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="pt-br" lang="pt-br" >
    <head>
    ================================================== -->
    <body>
<p style="text-align: justify;"><strong>VAGAS:</strong> 20 (14 na FUVEST + 6 no SiSU/ENEM)<br /><strong>DURA&Ccedil;&Atilde;O:</strong> 4 anos (integral)<br /><strong>COMO SE INSCREVER NA FUVEST:</strong> &nbsp;carreira 791 - curso 54</p>
<p><span style="background-color: initial;"></span></p>
<p>&nbsp;</p>
<p style="text-align: justify;"><strong>SOBRE O CURSO</strong></p>
<p style="text-align: justify;">O Bacharelado em Ci&ecirc;ncia de Dados visa formar um profissional capaz de &ldquo;pensar com dados&rdquo;. Diante da enorme quantidade de dados dispon&iacute;veis atualmente, provenientes das mais variadas fontes (como web, dispositivos m&oacute;veis, literaturas especializadas e sensores de todo tipo &ndash; como sat&eacute;lites, radares e at&eacute; telesc&oacute;pios), essa capacidade &eacute; essencial para a produ&ccedil;&at

Ao abrir o arquivo, já é possível notar que os nomes de entidades de caracteres incluídos no código HTML do site. Para que o texto fique mais condizente com a língua natural, serão feitas algumas substituições. É válido ressaltar que ao utilizar re.sub() a string original é alterada. Caso queira que sejam retornadas a string modificada e a string original, deve-se optar por .subn().


In [ ]:
aux_file = re.sub('\&ccedil;','ç',aux_file)
aux_file = re.sub('\&Ccedil;','Ç',aux_file)

aux_file = re.sub('\&aacute;','á',aux_file)
aux_file = re.sub('\&eacute;','é',aux_file)
aux_file = re.sub('\&iacute;','í',aux_file)
aux_file = re.sub('\&oacute;','ó',aux_file)
aux_file = re.sub('\&uacute;','ú',aux_file)

aux_file = re.sub('\&Aacute;','Á',aux_file)
aux_file = re.sub('\&Eacute;','É',aux_file)
aux_file = re.sub('\&Iacute;','Í',aux_file)
aux_file = re.sub('\&Oacute;','Ó',aux_file)
aux_file = re.sub('\&Uacute;','Ú',aux_file)

aux_file = re.sub('\&agrave;','à',aux_file)
aux_file = re.sub('\&Agrave;','À',aux_file)

aux_file = re.sub('\&atilde;','ã',aux_file)
aux_file = re.sub('\&otilde;','õ',aux_file)

aux_file = re.sub('\&Atilde;','Ã',aux_file)
aux_file = re.sub('\&Otilde;','Õ',aux_file)

aux_file = re.sub('\&acirc;','â',aux_file)
aux_file = re.sub('\&ecirc;','ê',aux_file)
aux_file = re.sub('\&ocirc;','ô',aux_file)

aux_file = re.sub('\&Acirc;','Â',aux_file)
aux_file = re.sub('\&Ecirc;','Ê',aux_file)
aux_file = re.sub('\&Ocirc;','Ô',aux_file)

aux_file = re.sub('\&ldquo;','"',aux_file)
aux_file = re.sub('\&rdquo;','"',aux_file)
aux_file = re.sub('\&ndash;','-',aux_file)
aux_file = re.sub('\&nbsp;',' ',aux_file)

print(aux_file)

<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="pt-br" lang="pt-br" >
    <head>
    ================================================== -->
    <body>
<p style="text-align: justify;"><strong>VAGAS:</strong> 20 (14 na FUVEST + 6 no SiSU/ENEM)<br /><strong>DURAÇÃO:</strong> 4 anos (integral)<br /><strong>COMO SE INSCREVER NA FUVEST:</strong>  carreira 791 - curso 54</p>
<p><span style="background-color: initial;"></span></p>
<p> </p>
<p style="text-align: justify;"><strong>SOBRE O CURSO</strong></p>
<p style="text-align: justify;">O Bacharelado em Ciência de Dados visa formar um profissional capaz de "pensar com dados". Diante da enorme quantidade de dados disponíveis atualmente, provenientes das mais variadas fontes (como web, dispositivos móveis, literaturas especializadas e sensores de todo tipo - como satélites, radares e até telescópios), essa capacidade é essencial para a produção de conhecimento novo, útil e relevante para apoio à decisão e resolução de problemas complexos da

As tags de HTML estão prejudicando a leitura do texto. Por isso, será feita a remoção delas. Existem bibliotecas que são capazes de efetuar a extração. No entanto, para fins didáticos, a alteração da string será feita com RegEx.

In [ ]:
aux_file = re.sub('\<br[ ]\/\>','\n',aux_file)
aux_file = re.sub('\<(\/)?([a-zA-Z0-9]+\=?\"?\-?\:?\;?\/?[ ]?)+(\=?\"?\-?\:?\;?\/?[ ]?)*\>','',aux_file)
# regex do site explicado na seção VERBOSE
aux_file = re.sub(r'(https?:(\/\/))?(www\.)?(\w+)\.(\w+)(\.?\/?)+([a-zA-Z0–9@:%&._\+-?!~#=]*)?(\.?\/)?([a-zA-Z0–9@:%&._\/+-~?!#=]*)?(\.?\/)?','',aux_file)
aux_file = re.sub('\<(\/)?(([a-zA-Z0-9]+\=?\"?\-?\:?\;?\/?[ ]?)+(\=?\"?\-?\:?\;?\/?[ ]?)*)*\>','',aux_file)
aux_file = re.sub('(\=?\<?\>?)*','',aux_file)
aux_file = re.sub('([  ]{2,})',' ',aux_file)
aux_file = re.sub('(\n[ ]*\n)','\n',aux_file)
aux_file = re.sub('(\n \n)','\n',aux_file)
aux_file = re.sub('^\n','',aux_file)
aux_file = re.sub('^ ','',aux_file)
aux_file = re.sub('\n$','',aux_file)

In [ ]:
print(aux_file)

--
VAGAS: 20 (14 na FUVEST + 6 no SiSU/ENEM)
DURAÇÃO: 4 anos (integral)
COMO SE INSCREVER NA FUVEST: carreira 791 - curso 54
SOBRE O CURSO
O Bacharelado em Ciência de Dados visa formar um profissional capaz de "pensar com dados". Diante da enorme quantidade de dados disponíveis atualmente, provenientes das mais variadas fontes (como web, dispositivos móveis, literaturas especializadas e sensores de todo tipo - como satélites, radares e até telescópios), essa capacidade é essencial para a produção de conhecimento novo, útil e relevante para apoio à decisão e resolução de problemas complexos da sociedade.
O QUE VOCÊ VAI ESTUDAR?
O curso é multidisciplinar e se encontra na junção das áreas de Computação, Estatística e Matemática. Essa graduação visa fornecer conhecimento conceitual sólido para a ciência de dados e habilidades práticas para a resolução de problemas reais das mais variadas áreas, desenvolvendo também habilidades de comunicação e o comportamento cidadão e ético, tão relevant

In [ ]:
file_ = open("final_html.txt", "x")
file_.write(aux_file)
file_.close()

# Verbose

É interessante usar VERBOSE para facilitar o entendimento de expressões regulares que possam ser mais longas. No caso, é recomendável compilar o padrão à parte antes de incluir dentro da função para substituição.

In [ ]:
string = 'https://www.google.com/'
print(string)

pattern = re.compile(r"""
(https?:(\/\/))? # verifica se o site inicia com http:// ou https://
(www\.)? # verifica se o site possui um trecho www.
(\w+)\. # uma sequência de caracteres alfanuméricos com um ponto em seguida
(\w+)   # uma sequência de caracteres alfanuméricos
(\.?\/?)+ # pelo menos uma vez ocorre um ponto . ou uma barra /
([a-zA-Z0–9@:%&._\+-?!~#=]*)? # nos colchetes: caracteres alfanuméricos ou os especiais especificados (@:%&._\+-?!~#=) podem ocorrer 0 ou mais vezes;
                              # nos parênteses: o que estava especificado dentro dos colchetes ocorre nenhuma ou uma vez
(\.?\/)? # nenhuma vez ou uma vez ocorre um ponto . ou uma barra /
([a-zA-Z0–9@:%&._\/+-~?!#=]*)? # nos colchetes: caracteres alfanuméricos ou os especiais especificados (@:%&._\+-?!~#=) podem ocorrer 0 ou mais vezes;
                               # nos parênteses: o que estava especificado dentro dos colchetes ocorre nenhuma ou uma vez
(\.?\/)? # nenhuma vez ou uma vez ocorre um ponto . ou uma barra / no término do site
""", re.VERBOSE)

string = re.sub(pattern,'site encontrado',string)

print(string)

https://www.google.com/
site encontrado
